# TODO
* Assess whether to proceed with functions or classes
* Better way to assign learning rate
* Better way to deal with stateful-ness

## Parameters

In [43]:
batch_size = 10
seq_length = 10
rnn_size   = 128
vocab_size = 5
save_dir_GAN = 'models_GAN'
vocab_file = 'simple_vocab.pkl'
model = 'lstm'
num_layers = 2
grad_clip = 5
real_input_file = 'real_reviews.txt'
fake_input_file = 'fake_reviews.txt'
num_epochs_dis = 1
data_dir = 'data/gan/'
dis_lr = 0.0001

## Helper Functions

In [2]:
def load_vocab(save_dir_GAN, vocab_file):
    '''Load vocabulary objects.
    
    Args:
        save_dir_GAN:  Directory containing vocab_files.
        vocab_file: Vocab file to use.
    '''
    with open(os.path.join(save_dir_GAN, vocab_file)) as f:
            chars, vocab = cPickle.load(f)
    return chars, vocab

# Construct GAN

In [4]:
import tensorflow as tf
from tensorflow.python.ops.nn import rnn_cell
from tensorflow.python.ops.nn import rnn
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import nn_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.framework import ops
from tensorflow.python.ops.nn import seq2seq 
# TODO: Eliminate depencence on seq2seq

tf.reset_default_graph()

def construct_gan():
    if model == 'rnn':
        cell_gen = rnn_cell.BasicRNNCell(rnn_size)
        cell_dis = rnn_cell.BasicRNNCell(rnn_size)
    elif model == 'gru':
        cell_gen = rnn_cell.GRUCell(rnn_size)
        cell_dis = rnn_cell.GRUCell(rnn_size)
    elif model == 'lstm':
        cell_gen = rnn_cell.BasicLSTMCell(rnn_size, state_is_tuple=False)
        cell_dis = rnn_cell.BasicLSTMCell(rnn_size, state_is_tuple=False)
    else:
        raise NotImplementedError('Model type not supported: {}'
                                  .format(model))

    # Initial indices.
    indices = []
    batch_indices = tf.constant(3, shape=[batch_size])
    
    # Targets for Generator are 1
    targets = tf.placeholder(tf.int32, [batch_size, seq_length])
        
    # Generator Portion of GAN.
    with tf.variable_scope('generator'):
        outputs_gen, logit_sequence = [], []
        cell_gen = rnn_cell.MultiRNNCell([cell_gen] * num_layers)
        state_gen = cell_gen.zero_state(batch_size, tf.float32)

        with tf.variable_scope('rnn'):
            softmax_w = tf.get_variable('softmax_w', [rnn_size, vocab_size])
            softmax_b = tf.get_variable('softmax_b', [vocab_size])
            embedding = tf.get_variable('embedding', [vocab_size, rnn_size])
            inp = tf.nn.embedding_lookup(embedding, batch_indices)
    
            for i in xrange(seq_length):
                indices.append(batch_indices)
                if i > 0:
                    tf.get_variable_scope().reuse_variables()
              
                # RNN.
                output_gen, state_gen = cell_gen(inp, state_gen)
                logits_gen = tf.nn.xw_plus_b(output_gen, softmax_w, 
                                             softmax_b)
                
                # Sampling.
                sample_op = tf.stop_gradient(Categorical(
                                            logits_gen).sample(n=1))
                batch_indices = tf.squeeze(sample_op)
                inp = tf.nn.embedding_lookup(embedding, batch_indices)                
                
                # Use Only Logit Sampled.
                one_hot = tf.stop_gradient(tf.one_hot(batch_indices,
                                                      depth = vocab_size,
                                                      dtype = tf.float32))
                logit_gen = one_hot * logits_gen
                logit_sequence.append(logit_gen)
                outputs_gen.append(output_gen)

        # Sampled indices
        sample_op = tf.pack(indices)
    
    # Discriminator Portion of GAN. 
    with tf.variable_scope('discriminator'):
        cell_dis = rnn_cell.MultiRNNCell([cell_dis] * num_layers)
        state_dis = cell_dis.zero_state(batch_size, tf.float32)

        with tf.variable_scope('rnn'):
            softmax_w_dis = tf.get_variable('softmax_w', [rnn_size, 2])
            softmax_b_dis = tf.get_variable('softmax_b', [2])
            embedding_dis = tf.get_variable('embedding', [vocab_size, rnn_size])            

            # Input sequence to Discriminator.
            inputs_dis = []
            for logit in logit_sequence:
                inputs_dis.append(tf.matmul(logit, embedding_dis))

            # RNN.
            assert len(inputs_dis) == len(outputs_gen)
            outputs_dis, last_state_dis = seq2seq.rnn_decoder(inputs_dis,
                state_dis, cell_dis, loop_function=None)

            # Predictions.
            probs, logits = [], []
            for output_dis in outputs_dis:
                logit = tf.nn.xw_plus_b(output_dis, softmax_w_dis, softmax_b_dis)
                prob = tf.nn.softmax(logit)
                logits.append(logit)
                probs.append(prob)
                
    with tf.name_scope('train'):
        gen_loss = seq2seq.sequence_loss_by_example(logits, 
                                                    tf.unpack(tf.transpose(targets)), 
                                                    tf.unpack(tf.transpose(tf.ones_like(targets, dtype=tf.float32))))

        gen_cost = tf.reduce_sum(gen_loss) / batch_size
        tf.scalar_summary('training loss', gen_cost)
        lr_gen = tf.Variable(0.0, trainable = False)
        tvars = tf.trainable_variables()
        gen_vars = [v for v in tvars if not v.name.startswith("discriminator/")]

        gen_grads            = tf.gradients(gen_cost, gen_vars)
        all_grads            = tf.gradients(gen_cost, tvars)
        gen_grads_clipped, _ = tf.clip_by_global_norm(gen_grads, grad_clip)
        gen_optimizer        = tf.train.AdamOptimizer(lr_gen)
        gen_train_op         = gen_optimizer.apply_gradients(zip(gen_grads_clipped, gen_vars))
     
    return gen_train_op, sample_op

# Construct Discriminator

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.python.ops.nn import rnn_cell
from tensorflow.python.ops.nn import rnn
from tensorflow.python.ops.nn import seq2seq
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import nn_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.framework import ops

tf.reset_default_graph()

def construct_discriminator(input_data, targets):
    if model == 'rnn':
        cell = rnn_cell.BasicRNNCell(rnn_size)
    elif model == 'gru':
        cell = rnn_cell.GRUCell(rnn_size)
    elif model == 'lstm':
        cell = rnn_cell.BasicLSTMCell(rnn_size)
    else:
        raise Exception('model type not supported: {}'.format(model))

    cell = rnn_cell.MultiRNNCell([cell] * num_layers)
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    with tf.variable_scope('rnn'):
        softmax_w = tf.get_variable('softmax_w', [rnn_size, 2])
        softmax_b = tf.get_variable('softmax_b', [2])
        embedding = tf.get_variable('embedding', [vocab_size, rnn_size])
        inputs    = tf.split(1, seq_length, tf.nn.embedding_lookup(embedding, input_data))
        inputs    = [tf.squeeze(i, [1]) for i in inputs]

        outputs, final_state = seq2seq.rnn_decoder(inputs, initial_state, 
            cell, loop_function=None)

    output_tf = tf.reshape(tf.concat(1, outputs), [-1, rnn_size])
    logits = tf.nn.xw_plus_b(output_tf, softmax_w, softmax_b)
    probs  = tf.nn.softmax(logits)

    loss = seq2seq.sequence_loss_by_example(
        [logits],
        [tf.reshape(targets, [-1])],
        [tf.ones([batch_size * seq_length])])

    cost = tf.reduce_sum(loss) / batch_size / seq_length

    lr          = tf.Variable(0.0, trainable = False)
    tvars       = tf.trainable_variables()
    grads, _    = tf.clip_by_global_norm(tf.gradients(cost, tvars, aggregation_method=2), grad_clip)
    optimizer   = tf.train.AdamOptimizer(lr)
    train_op    = optimizer.apply_gradients(zip(grads, tvars))
    
    return train_op, cost, lr

# Train NLP GAN.
Train the NLP GAN by using the two defined networks.


In [49]:
from batcher_gan import DiscriminatorBatcher, GANBatcher
from tensorflow.contrib.distributions import Categorical
import time

tf.reset_default_graph()


# def reset_reviews(data_dir, file_name):
#     '''Clear the file containing the generated reviews.
    
#     Args:
#         data_dir:  Directory to store generated reviews.
#         file_name:  Name of file containing generated reviews.
#     '''
#     print('Reseting reviews at %s' % os.path.join(data_dir, file_name))
#     open(os.path.join(data_dir, file_name), 'w').close()
   


def generate_samples(sess, save_dir, data_file, sample_op, chars):
    '''Generate samples.'''
    data_file = os.path.join(save_dir, data_file)
    indices = sess.run(sample_op)
    int_to_char = lambda x: chars[x]
    mapfunc = np.vectorize(int_to_char)
    samples = mapfunc(indices.T)
    print('Generating samples to %s' % data_file)    
    with open(data_file, 'a+') as f:
        for line in samples:
            print line
            print>>f, ''.join(line) 
                  

# Generator graph.
gen_train_op, sample_op = construct_gan()

# Discriminator graph.
input_data = tf.placeholder(tf.int32, [batch_size, seq_length])
targets    = tf.placeholder(tf.int32, [batch_size, seq_length])
dis_train_op, dis_cost, lr = construct_discriminator(input_data, targets)
                            
with tf.Session() as sess:
    # Graph initialization.
    init_op = tf.initialize_all_variables()
    sess.run(init_op)
        
    # Clear old samples.
    reset_reviews(save_dir_GAN, fake_input_file)
    
    # Generate new samples.
    chars, vocab = load_vocab(save_dir_GAN, vocab_file)
    generate_samples(sess, save_dir_GAN, fake_input_file, sample_op, chars)
    batcher  = DiscriminatorBatcher(real_input_file, 
                                        fake_input_file, 
                                        data_dir, vocab_file,
                                        batch_size, seq_length)
    batcher.reset_batch_pointer()
    
    # Training Discriminator.
    for i in xrange(200):
        x, y  = batcher.next_batch()
#         print 'x', x
#         print 'y', y
        
        # TODO: Why use assign vs. a feed?
        # Best approach for this?
#         sess.run(tf.assign(lr, dis_lr))
        feed = {input_data: x, targets: y, lr: dis_lr}
        cost, _ = sess.run([dis_cost, dis_train_op], feed)
        
        print('Loss for iteration %i: %f' % (i, cost))


Reseting reviews at models_GAN/fake_reviews.txt
Generating samples to models_GAN/fake_reviews.txt
[' ' ' ' ' ' '\n' ' ' ' ' ' ' ' ' 'b' ' ']
[' ' ' ' ' ' ' ' ' ' '\n' '\n' ' ' ' ' 'e']
[' ' ' ' '\n' '\n' ' ' '\n' ' ' ' ' 'e' ' ']
[' ' 'b' '\n' ' ' 'b' ' ' 'b' '\n' ' ' ' ']
[' ' 'e' ' ' ' ' 'r' ' ' ' ' ' ' ' ' ' ']
[' ' ' ' '\n' ' ' 'r' ' ' ' ' 'b' '\n' '\n']
[' ' 'b' 'b' ' ' ' ' 'e' ' ' ' ' ' ' 'r']
[' ' '\n' ' ' ' ' ' ' ' ' ' ' '\n' '\n' '\n']
[' ' ' ' 'b' 'e' ' ' '\n' ' ' ' ' '\n' ' ']
[' ' ' ' ' ' ' ' '\n' ' ' '\n' ' ' ' ' 'r']
Loss for iteration 0: 0.799852
Loss for iteration 1: 0.786291
Loss for iteration 2: 0.780016
Loss for iteration 3: 0.774751
Loss for iteration 4: 0.768128
Loss for iteration 5: 0.760939
Loss for iteration 6: 0.755057
Loss for iteration 7: 0.748485
Loss for iteration 8: 0.738555
Loss for iteration 9: 0.738642
Loss for iteration 10: 0.732029
Loss for iteration 11: 0.721299
Loss for iteration 12: 0.714165
Loss for iteration 13: 0.707675
Loss for iteration 14: 0.

# Generate

In [10]:
import tensorflow as tf
import os
import cPickle
from tensorflow.contrib.distributions import Categorical
tf.reset_default_graph()


def map_to_text(indices, chars):
    '''Return text from indices'''
    text = []
    for index in indices:
        text.append(chars[index])
    return text


def generate():    
    # Initial indices
    batch_indices = tf.constant(0, shape=[batch_size])

    # RNN
    outputs, indices = [], []
    cell = tf.nn.rnn_cell.BasicRNNCell(rnn_size)
    state = cell.zero_state(batch_size, tf.float32)
        
    with tf.variable_scope('rnn'):
        # Embeddings and Logits
        embedding = tf.get_variable('embedding', [vocab_size, rnn_size])
        softmax   = tf.get_variable('softmax', [rnn_size, vocab_size])

        inp = tf.nn.embedding_lookup(embedding, batch_indices)
        for i in xrange(seq_length):
            indices.append(batch_indices)
            if i > 0:
                tf.get_variable_scope().reuse_variables()
            rnn_out, state = cell(inp, state)
            logits_out = tf.matmul(rnn_out, softmax)
            outputs.append(logits_out)
            batch_indices = tf.squeeze(Categorical(logits_out).sample(n=1))
            inp = tf.nn.embedding_lookup(embedding, batch_indices)
  
    return outputs, indices
            

with tf.Session() as sess: 
    outputs, indices = generate()
    init_op = tf.initialize_all_variables()
    sess.run(init_op)

    chars, vocab = load_vocab(save_dir_GAN, vocab_file)
    
    for i, output in enumerate(outputs):
        print 'Iteration %d'%i
        print sess.run(output),'\n'
    
    for i, batch_indices in enumerate(indices):
        print 'Iteration %d'%i
        indices_eval = sess.run(batch_indices)
        print map_to_text(indices_eval, chars),'\n'
    
#     for line in indices:
#         print ''.join(line)

Iteration 0
[[-0.47268498 -0.29608265 -0.6129449   0.68132377  0.20629463]
 [-0.47268498 -0.29608265 -0.6129449   0.68132377  0.20629463]] 

Iteration 1
[[ 0.13426088 -0.31010672 -0.1337792  -0.00184738  0.48531109]
 [-0.2791602  -0.23040935 -0.2588793   0.36823389  0.14612493]] 

Iteration 2
[[-0.46070901 -0.16181867 -0.94683945  0.83026254  0.16507755]
 [-0.37143287 -0.02318827  0.17404538 -0.07132882 -0.11844563]] 

Iteration 3
[[-0.46001613 -0.20393686 -0.96491438  0.79266554  0.1573296 ]
 [-0.40352771  0.12216403  0.66109788 -0.24784262  0.14009601]] 

Iteration 0
['e', 'e'] 

Iteration 1
['b', '\n'] 

Iteration 2
['\n', 'b'] 

Iteration 3
[' ', ' '] 



## Random

In [90]:
def variable_summaries(var, name):
    '''Attach a lot of summaries to a Tensor.'''
    mean = tf.reduce_mean(var)
    tf.scalar_summary('mean/' + name, mean)
    with tf.name_scope('stddev'):
        stddev = tf.sqrt(tf.reduce_sum(tf.square(var - mean)))
    tf.scalar_summary('sttdev/' + name, stddev)
    tf.scalar_summary('max/' + name, tf.reduce_max(var))
    tf.scalar_summary('min/' + name, tf.reduce_min(var))
    tf.histogram_summary(name, var)